In [2]:
import pandas as pd
import os
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx
import bokeh
import os
import openai
from openai import OpenAI
import backoff
import numpy as np
from sklearn.cluster import KMeans
import backoff
import requests 

In [3]:
# import csv with 'phenotype' descriptor column

file = 'Loinc.csv' 
df = pd.read_csv(file)

print(df.head()) 


  LOINC_NUM                                          COMPONENT PROPERTY  \
0  100000-9  Health informatics pioneer and the father of L...       Hx   
1  100001-7  Health informatics pioneer and cofounder of LOINC       Hx   
2  100002-5                        Specimen care is maintained     Find   
3  100003-3   Team communication is maintained throughout care     Find   
4  100004-1  Demonstrates knowledge of the expected psychos...     Find   

  TIME_ASPCT    SYSTEM SCALE_TYP METHOD_TYP        CLASS VersionLastChanged  \
0         Pt  ^Patient       Nar        NaN       H&P.HX               2.74   
1         Pt  ^Patient       Nar        NaN       H&P.HX               2.74   
2         Pt  ^Patient       Ord        NaN  SURVEY.PNDS               2.72   
3         Pt  ^Patient       Ord        NaN  SURVEY.PNDS               2.72   
4         Pt  ^Patient       Ord        NaN  SURVEY.PNDS               2.72   

  CHNG_TYPE  ... COMMON_TEST_RANK COMMON_ORDER_RANK  HL7_ATTACHMENT_STRUCT

/var/folders/pq/qx5x9fj10pj3v7hdcsd6qzz40000gn/T/ipykernel_93246/395550858.py:4: DtypeWarning: Columns (8,15,22,35,37,38) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


In [4]:
df

,LOINC_NUM,COMPONENT,PROPERTY,TIME_ASPCT,SYSTEM,SCALE_TYP,METHOD_TYP,CLASS,VersionLastChanged,CHNG_TYPE,...,COMMON_TEST_RANK,COMMON_ORDER_RANK,HL7_ATTACHMENT_STRUCTURE,EXTERNAL_COPYRIGHT_LINK,PanelType,AskAtOrderEntry,AssociatedObservations,VersionFirstReleased,ValidHL7AttachmentRequest,DisplayName
0,100000-9,Health informatics pioneer and the father of L...,Hx,Pt,^Patient,Nar,NaN,H&P.HX,2.74,ADD,...,0,0,NaN,NaN,NaN,NaN,NaN,2.74,NaN,NaN
1,100001-7,Health informatics pioneer and cofounder of LOINC,Hx,Pt,^Patient,Nar,NaN,H&P.HX,2.74,ADD,...,0,0,NaN,NaN,NaN,NaN,NaN,2.74,NaN,NaN
2,100002-5,Specimen care is maintained,Find,Pt,^Patient,Ord,NaN,SURVEY.PNDS,2.72,ADD,...,0,0,NaN,NaN,NaN,NaN,NaN,2.72,NaN,NaN
3,100003-3,Team communication is maintained throughout care,Find,Pt,^Patient,Ord,NaN,SURVEY.PNDS,2.72,ADD,...,0,0,NaN,NaN,NaN,NaN,NaN,2.72,NaN,NaN
4,100004-1,Demonstrates knowledge of the expected psychos...,Find,Pt,^Patient,Ord,NaN,SURVEY.PNDS,2.72,ADD,...,0,0,NaN,NaN,NaN,NaN,NaN,2.72,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102460,99994-6,"Fluid, electrolyte, and acid-base balances are...",Find,Pt,^Patient,Ord,NaN,SURVEY.PNDS,2.72,ADD,...,0,0,NaN,NaN,NaN,NaN,NaN,2.72,NaN,NaN
102461,99995-3,Respiratory status is maintained at or improve...,Find,Pt,^Patient,Ord,NaN,SURVEY.PNDS,2.72,ADD,...,0,0,NaN,NaN,NaN,NaN,NaN,2.72,NaN,NaN
102462,99996-1,Cardiovascular status is maintained at or impr...,Find,Pt,^Patient,Ord,NaN,SURVEY.PNDS,2.72,ADD,...,0,0,NaN,NaN,NaN,NaN,NaN,2.72,NaN,NaN
102463,99997-9,Demonstrates &or reports adequate pain control,Find,Pt,^Patient,Ord,NaN,SURVEY.PNDS,2.72,ADD,...,0,0,NaN,NaN,NaN,NaN,NaN,2.72,NaN,NaN


In [5]:
#define API from global variable

openai.api_key = os.environ.get("OPENAI_API_KEY")

In [6]:
# try and catch to call OpenAI API

@backoff.on_exception(backoff.expo, requests.exceptions.RequestException, max_tries=8)
def get_embedding(text, model="text-embedding-ada-002"):
    text = text.replace("\n", " ")
    try:
        return openai.embeddings.create(input=[text], model=model).data[0].embedding
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        raise  # Re-raise the exception to trigger backoff

        

In [7]:
# Trying get_embeddings for just one row
first_row_embedding = get_embedding(df['COMPONENT'].iloc[0])

# Create a new column 'embeddings' and set the first value
# Initialize the column with None or np.nan for the remaining entries
df['embeddings'] = None
# Set the embedding for the first row
df.at[0, 'embeddings'] = first_row_embedding



In [ ]:

df['embeddings'] = df['COMPONENT'].apply(get_embedding)

In [ ]:
#df.to_pickle('phecode_gpt_embedding.pkl')
df.to_pickle('LOINC_gpt_embedding.pkl')